# Connection à Mariadb


In [ ]:
import mariadb
import json
from pathlib import Path
import time

# --- Configuration ---
DB_NAME = "music_data"
DATA_DIR = "."  # répertoire des fichiers JSON
BATCH_SIZE = 1000  # nombre d'insertions par lot

# --- Connexion MariaDB ---
def connect_db():
    try:
        conn = mariadb.connect(
            user="root",
            password="root",
            host="localhost",
            port=3307,
            autocommit=False
        )
        print("✅ Connexion réussie à MariaDB !")
        return conn
    except mariadb.Error as e:
        print(f"❌ Erreur de connexion : {e}")
        return None

# --- Création du schéma selon la nouvelle structure ---
def setup_database(conn):
    cursor = conn.cursor()
    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME};")
        cursor.execute(f"USE {DB_NAME};")

        # Table infos
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS infos (
                slice VARCHAR(255) PRIMARY KEY,
                generated_on DATETIME,
                version VARCHAR(50)
            );
        """)

        # Table artists
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS artists (
                artist_uri VARCHAR(255) PRIMARY KEY,
                artist_name VARCHAR(255)
            );
        """)

        # Table albums
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS albums (
                album_uri VARCHAR(255) PRIMARY KEY,
                album_name VARCHAR(255)
            );
        """)

        # Table tracks
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS tracks (
                track_uri VARCHAR(255) PRIMARY KEY,
                track_name VARCHAR(255),
                duration_ms INT,
                album_uri VARCHAR(255),
                artist_uri VARCHAR(255)
            );
        """)

        # Table playlists
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS playlists (
                pid INT PRIMARY KEY,
                name VARCHAR(255),
                collaborative BOOLEAN,
                modified_at DATETIME,
                duration_ms INT,
                num_tracks INT,
                num_albums INT,
                num_artists INT,
                num_followers INT,
                num_edits INT,
                info_slice VARCHAR(255)
            );
        """)

        # Table playlists_tracks
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS playlists_tracks (
                playlist_id INT,
                track_uri VARCHAR(255),
                pos INT,
                PRIMARY KEY (playlist_id, track_uri, pos),
                UNIQUE (playlist_id, pos),
                UNIQUE (playlist_id, track_uri)
            );
        """)

        conn.commit()
        print(f"✅ Base de données '{DB_NAME}' et tables prêtes.")
    except mariadb.Error as e:
        print(f"❌ Erreur création tables : {e}")
        conn.rollback()
    finally:
        cursor.close()

# --- Insertion batch générique ---
def insert_batch(cursor, table, columns, values):
    if not values:
        return
    placeholders = ','.join(['?' for _ in columns])
    cols = ','.join(columns)
    try:
        cursor.executemany(f"INSERT IGNORE INTO {table} ({cols}) VALUES ({placeholders});", values)
    except mariadb.Error as e:
        print(f"❌ Erreur insertion dans {table}: {e}")

# --- Traitement d’un fichier JSON ---
def process_json_file(file_path, conn, all_artists, all_albums, all_tracks, all_playlists_tracks):
    cursor = conn.cursor()
    start = time.time()

    playlists_to_insert = []
    infos_to_insert = []

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        playlists = data.get("playlists", [])
        info_data = data.get("info", {})

        # Insérer info slice
        slice_name = info_data.get("slice", Path(file_path).stem)
        infos_to_insert.append((
            slice_name,
            info_data.get("generated_on"),
            info_data.get("version")
        ))

        print(f"📄 Traitement du fichier {file_path.name} ({len(playlists)} playlists)...")

        for playlist in playlists:
            playlists_to_insert.append((
                playlist.get("pid"),
                playlist.get("name", "Untitled Playlist"),
                1 if playlist.get("collaborative", False) else 0,
                playlist.get("modified_at"),
                playlist.get("duration_ms"),
                playlist.get("num_tracks"),
                playlist.get("num_albums"),
                playlist.get("num_artists"),
                playlist.get("num_followers"),
                playlist.get("num_edits"),
                slice_name
            ))

            for track in playlist.get("tracks", []):
                track_uri = track.get("track_uri")
                if not track_uri:
                    continue

                artist_uri = track.get("artist_uri")
                album_uri = track.get("album_uri")
                artist_name = track.get("artist_name", "").strip()
                album_name = track.get("album_name", "").strip()

                # Collecte artistes / albums / pistes uniques
                if artist_uri and artist_uri not in all_artists:
                    all_artists[artist_uri] = artist_name
                if album_uri and album_uri not in all_albums:
                    all_albums[album_uri] = album_name
                if track_uri not in all_tracks:
                    all_tracks[track_uri] = {
                        "track_uri": track_uri,
                        "track_name": track.get("track_name", "Unknown Track"),
                        "duration_ms": track.get("duration_ms"),
                        "album_uri": album_uri,
                        "artist_uri": artist_uri
                    }

                # playlist_tracks avec position
                pos = track.get("pos", 0)
                all_playlists_tracks.append((playlist.get("pid"), track_uri, pos))

        # Insertion infos et playlists
        insert_batch(cursor, "infos", ["slice", "generated_on", "version"], infos_to_insert)
        insert_batch(cursor, "playlists", [
            "pid", "name", "collaborative", "modified_at", "duration_ms",
            "num_tracks", "num_albums", "num_artists", "num_followers",
            "num_edits", "info_slice"
        ], playlists_to_insert)

        conn.commit()
        print(f"✅ {file_path.name} traité en {time.time() - start:.2f}s")

    except Exception as e:
        print(f"❌ Erreur {file_path}: {e}")
        conn.rollback()
    finally:
        cursor.close()

# --- Pipeline principal ---
def main():
    conn = connect_db()
    if not conn:
        return

    setup_database(conn)

    data_path = Path(DATA_DIR)
    json_files = sorted(list(data_path.glob("mpd.slice.*.json")))

    if not json_files:
        print(f"⚠️ Aucun fichier JSON trouvé dans '{DATA_DIR}'.")
        conn.close()
        return

    all_artists = {}
    all_albums = {}
    all_tracks = {}
    all_playlists_tracks = []

    total_start = time.time()

    # Lecture des fichiers
    for file_path in json_files:
        process_json_file(file_path, conn, all_artists, all_albums, all_tracks, all_playlists_tracks)

    cursor = conn.cursor()

    # Insertion artistes
    print(f"🎤 Insertion de {len(all_artists)} artistes...")
    insert_batch(cursor, "artists", ["artist_uri", "artist_name"], [(k, v) for k, v in all_artists.items()])

    # Insertion albums
    print(f"💿 Insertion de {len(all_albums)} albums...")
    insert_batch(cursor, "albums", ["album_uri", "album_name"], [(k, v) for k, v in all_albums.items()])

    # Insertion tracks
    print(f"🎧 Insertion de {len(all_tracks)} pistes...")
    insert_batch(cursor, "tracks", ["track_uri", "track_name", "duration_ms", "album_uri", "artist_uri"],
                 [(t["track_uri"], t["track_name"], t["duration_ms"], t["album_uri"], t["artist_uri"])
                  for t in all_tracks.values()])

    # Insertion playlists_tracks
    print(f"📚 Insertion de {len(all_playlists_tracks)} associations playlist–piste...")
    insert_batch(cursor, "playlists_tracks", ["playlist_id", "track_uri", "pos"], all_playlists_tracks)

    conn.commit()
    conn.close()

    print(f"✅ Import complet en {time.time() - total_start:.2f}s.")

if __name__ == "__main__":
    main()
